# FinalEx

### Introduction:

This time you will create the data.



### Step 1. Import the necessary libraries

In [57]:
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### Step 2. 

In [58]:
df_1 = pd.read_csv('./product_data/product_raw_data_v2.csv') 
df_2 = pd.read_csv('./product_data/transaction_raw_data_v3.csv') 
df_err_1 = pd.read_csv('error_data_part1.csv')
df_err_2 =pd.read_csv('error_data_part2.csv')
df_pressure_data = pd.read_csv('./pressure_data/pressure_data.csv')
df = pd.concat([df_1 , df_2])
df_error = pd.concat([df_err_1,df_err_2])
df.drop(['Unnamed: 0'],axis=1 , inplace = True) 
def check_out_time(x):
        return x[0 : 7]

df['check_out_time'] = df['out_time'].apply(check_out_time)
def check_in_time(x):
        return x[0 : 7]

df['check_in_time'] = df['in_time'].apply(check_in_time) # tạo thêm cột check_out_time và check_in_time để gom nhóm kiểm tra
df['out_time'] = pd.to_datetime(df['out_time'] , format='%Y-%m-%d %H-%M-%S')
df['in_time'] = pd.to_datetime(df['in_time'] , format='%Y-%m-%d %H-%M-%S')
df = df.sort_values(by=['in_time'])
# df_machine_001 = df[df['machine_name'] == 'MACHINE_001']
# df_machine_002 = df[df['machine_name'] == 'MACHINE_002']
# df_machine_001
df = df.merge(df_error , on='product_id' , how='left')
df


,machine_name,out_time,in_time,product_id,check_out_time,check_in_time,Unnamed: 0,errors,code
0,MACHINE_002,2014-12-01 22:02:35,2014-12-01 21:28:45,PRODUCT_10010,2014-12,2014-12,NaN,NaN,NaN
1,MACHINE_001,2014-12-01 22:52:50,2014-12-01 21:54:21,PRODUCT_13,2014-12,2014-12,NaN,NaN,NaN
2,MACHINE_002,2014-12-01 22:52:56,2014-12-01 22:02:43,PRODUCT_10011,2014-12,2014-12,NaN,NaN,NaN
3,MACHINE_002,2014-12-01 23:39:59,2014-12-01 22:53:08,PRODUCT_10012,2014-12,2014-12,NaN,NaN,NaN
4,MACHINE_001,2014-12-01 23:54:18,2014-12-01 22:54:34,PRODUCT_1,2014-12,2014-12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10666,MACHINE_002,2015-07-11 18:21:18,2015-07-11 16:46:23,PRODUCT_14849,2015-07,2015-07,NaN,NaN,NaN
10667,MACHINE_001,2015-07-11 17:40:37,2015-07-11 17:01:46,PRODUCT_5594,2015-07,2015-07,NaN,NaN,NaN
10668,MACHINE_001,2015-07-11 18:32:18,2015-07-11 17:40:50,PRODUCT_5604,2015-07,2015-07,NaN,NaN,NaN
10669,MACHINE_002,2015-07-11 20:50:40,2015-07-11 19:14:34,PRODUCT_14856,2015-07,2015-07,NaN,NaN,NaN


### Step 3. 

In [59]:
df_pressure_data.drop(['Unnamed: 0'],axis=1 , inplace = True)
def check_time(x):
        return x[0 : 7] 
df_pressure_data['check_time'] = df_pressure_data['start_at'].apply(check_time) # tạo ra cột mới để chứa ngày tháng năm tiện cho việc lọc
def fix_unsynchronize(x):
    x = x - datetime.timedelta(minutes=1)
    return x
df_pressure_data['start_at'] = pd.to_datetime(df_pressure_data['start_at'] ,format='%Y-%m-%d %H:%M:%S')
df_pressure_data['start_at'] =df_pressure_data['start_at'].apply(fix_unsynchronize)
df_pressure_data = df_pressure_data.sort_values(by=['start_at'])
arr_time_unique = list(pd.unique(df_pressure_data['check_time']))# tạo ra mảng chứa các ngày riêng biệt 
print(arr_time_unique)
df_pressure_data

['2014-12', '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07']


,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_196,pressure_197,pressure_198,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,check_time
0,0.0,2.639808,4.771886,5.979325,6.682468,7.285566,7.206227,8.655227,10.800956,10.964720,...,196.880196,197.181910,199.199552,199.762799,201.182017,201.180853,2014-12-10 00:09:50,Sub1,File_1,2014-12
1,0.0,2.795488,3.251834,4.499492,5.502105,6.449215,8.087633,9.946180,10.219085,10.743145,...,197.285807,197.214208,199.259451,199.469542,201.953278,201.091001,2014-12-10 00:15:28,Sub1,File_2,2014-12
8,0.0,2.989949,3.731346,5.592048,6.855218,7.751051,7.899846,8.706237,9.754890,11.872049,...,196.039824,198.628021,199.712650,199.619340,200.246648,202.049603,2014-12-10 00:16:05,Sub2,File_9,2014-12
2,0.0,3.807030,3.503354,4.655089,6.302633,7.726263,8.249255,9.675065,9.591786,10.676826,...,197.783402,197.323749,198.747466,199.035431,200.608281,201.498844,2014-12-10 00:20:41,Sub1,File_3,2014-12
9,0.0,3.740250,4.023481,5.676871,6.898762,6.879299,8.869699,9.950092,9.061035,11.285521,...,197.012501,197.445732,198.990163,200.082805,201.440099,201.270897,2014-12-10 00:21:42,Sub2,File_10,2014-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186755,0.0,2.931815,3.039504,5.962543,6.046933,6.575745,7.143761,8.365927,9.133804,10.985532,...,196.830091,197.910570,198.944525,199.756017,201.094636,201.726737,2015-07-11 18:03:50,Sub2,File_199504,2015-07
186770,0.0,3.630130,4.978932,4.390446,5.711679,6.356601,8.754259,9.606328,9.658834,11.409551,...,196.632754,198.333930,198.997033,199.810128,201.865581,202.114220,2015-07-11 18:04:28,Sub3,File_199518,2015-07
186787,0.0,2.394231,3.119259,5.783146,6.586384,6.113767,7.774616,8.288652,10.241900,11.189193,...,197.438755,198.728193,199.458638,200.994531,201.110963,201.023481,2015-07-11 18:05:47,Sub4,File_199530,2015-07
186754,0.0,3.855237,3.817676,5.714153,5.665591,7.170642,7.624416,8.157029,10.176965,10.818476,...,196.575722,198.553787,198.456641,199.286844,200.238973,202.587612,2015-07-11 18:06:25,Sub1,File_199499,2015-07


### Step 4. 

In [60]:
df_pressure_2014_12 =  df_pressure_data[df_pressure_data['check_time'] == arr_time_unique[0]] # tách từng tháng để kiểm tra
df_product_2014_12 = df[(df['check_out_time'] == arr_time_unique[0]) | (df['check_in_time'] == arr_time_unique[0])]
df_pressure_2014_12['product_id'] = '####'
df_pressure_2014_12['error_code'] = '####'
df_pressure_2014_12['error_quantity'] = '####'
for index in range(len(df_pressure_2014_12)) :
        df_product_2014_12_find = df_product_2014_12[(df_product_2014_12['out_time'] >= df_pressure_2014_12.iloc[index , 201]) & (df_product_2014_12['in_time'] <= df_pressure_2014_12.iloc[index , 201])]
        df_pressure_2014_12.iloc[index , 205] ='-'.join(list(set(df_product_2014_12_find['product_id'].values)))
        error_code = [x for x in df_product_2014_12_find['code'] if str(x) != 'nan']
        df_pressure_2014_12.iloc[index , 206] ='-'.join(error_code)
        error_quantity = [x for x in df_product_2014_12_find['errors'] if str(x) != 'nan']
        total_quantity = sum(error_quantity)
        df_pressure_2014_12.iloc[index , 207] = total_quantity
df_pressure_2014_12


<ipython-input-60-d36f3d1e2d26>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2014_12['product_id'] = '####'
<ipython-input-60-d36f3d1e2d26>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2014_12['error_code'] = '####'
<ipython-input-60-d36f3d1e2d26>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,check_time,product_id,error_code,error_quantity
0,0.0,2.639808,4.771886,5.979325,6.682468,7.285566,7.206227,8.655227,10.800956,10.964720,...,199.762799,201.182017,201.180853,2014-12-10 00:09:50,Sub1,File_1,2014-12,PRODUCT_141-PRODUCT_10165,BPAK,5.0
1,0.0,2.795488,3.251834,4.499492,5.502105,6.449215,8.087633,9.946180,10.219085,10.743145,...,199.469542,201.953278,201.091001,2014-12-10 00:15:28,Sub1,File_2,2014-12,PRODUCT_141-PRODUCT_10165,BPAK,5.0
8,0.0,2.989949,3.731346,5.592048,6.855218,7.751051,7.899846,8.706237,9.754890,11.872049,...,199.619340,200.246648,202.049603,2014-12-10 00:16:05,Sub2,File_9,2014-12,PRODUCT_141-PRODUCT_10165,BPAK,5.0
2,0.0,3.807030,3.503354,4.655089,6.302633,7.726263,8.249255,9.675065,9.591786,10.676826,...,199.035431,200.608281,201.498844,2014-12-10 00:20:41,Sub1,File_3,2014-12,PRODUCT_141-PRODUCT_10165,BPAK,5.0
9,0.0,3.740250,4.023481,5.676871,6.898762,6.879299,8.869699,9.950092,9.061035,11.285521,...,200.082805,201.440099,201.270897,2014-12-10 00:21:42,Sub2,File_10,2014-12,PRODUCT_141-PRODUCT_10165,BPAK,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147941,0.0,3.140372,4.469120,4.405584,6.345092,6.147697,8.345390,8.111260,9.637910,10.714389,...,199.079076,200.895493,201.981407,2014-12-31 23:54:45,Sub1,File_158257,2014-12,PRODUCT_648-PRODUCT_10712,EMAN,4.0
148015,0.0,3.455011,3.824269,5.036430,6.777424,6.676958,8.066350,8.077175,9.338047,10.181505,...,199.940438,201.350138,201.045617,2014-12-31 23:55:45,Sub2,File_158345,2014-12,PRODUCT_648-PRODUCT_10712,EMAN,4.0
148132,0.0,2.761433,4.652734,5.017049,5.727028,7.490534,8.875852,9.479491,9.241258,11.166491,...,200.456250,200.831983,201.990349,2014-12-31 23:56:44,Sub3,File_158452,2014-12,PRODUCT_648-PRODUCT_10712,EMAN,4.0
148239,0.0,2.497000,3.544414,5.723406,5.001954,7.851735,8.478232,9.998531,10.758066,10.775182,...,200.840668,200.681485,201.123626,2014-12-31 23:57:40,Sub4,File_158571,2014-12,PRODUCT_648-PRODUCT_10712,EMAN,4.0


In [61]:
df_pressure_2015_01 =  df_pressure_data[df_pressure_data['check_time'] == arr_time_unique[1]]
df_product_2015_01 = df[(df['check_out_time'] == arr_time_unique[1]) | (df['check_in_time'] == arr_time_unique[1])]
df_pressure_2015_01['product_id'] = '####'
df_pressure_2015_01['error_code'] = '####'
df_pressure_2015_01['error_quantity'] = '####'
for index in range(len(df_pressure_2015_01)) :
        df_product_2015_01_find = df_product_2015_01[(df_product_2015_01['out_time'] >= df_pressure_2015_01.iloc[index , 201]) & (df_product_2015_01['in_time'] <= df_pressure_2015_01.iloc[index , 201])]
        df_pressure_2015_01.iloc[index , 205] ='-'.join(list(df_product_2015_01_find['product_id'].values)) 
        error_code = [x for x in df_product_2015_01_find['code'] if str(x) != 'nan']
        df_pressure_2015_01.iloc[index , 206] ='-'.join(error_code)
        error_quantity = [x for x in df_product_2015_01_find['errors'] if str(x) != 'nan']
        total_quantity = sum(error_quantity)
        df_pressure_2015_01.iloc[index , 207] = total_quantity
            
df_pressure_2015_01

<ipython-input-61-75d6da4496b2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_01['product_id'] = '####'
<ipython-input-61-75d6da4496b2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_01['error_code'] = '####'
<ipython-input-61-75d6da4496b2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,check_time,product_id,error_code,error_quantity
148292,0.0,2.796094,3.934605,5.495417,6.555060,7.928752,7.086302,9.882418,10.842944,11.866157,...,200.441845,200.169169,201.089014,2015-01-01 00:00:20,Sub2,File_158630,2015-01,PRODUCT_648-PRODUCT_10712,EMAN,4.0
148397,0.0,3.036180,3.627198,5.345046,6.344784,7.697749,8.272049,8.720561,9.069317,10.486234,...,200.747442,200.581401,202.509313,2015-01-01 00:01:20,Sub3,File_158736,2015-01,PRODUCT_648-PRODUCT_10712,EMAN,4.0
148249,0.0,3.065631,4.533843,4.549771,5.439306,6.623906,7.948082,9.089998,10.326669,10.692485,...,199.049813,201.315940,201.214456,2015-01-01 00:10:10,Sub1,File_158574,2015-01,PRODUCT_10712-PRODUCT_634,EMAN,4.0
148308,0.0,3.482146,4.339498,5.226784,5.827035,6.866303,8.919291,9.295008,9.217480,11.282774,...,199.689907,200.185174,202.162756,2015-01-01 00:10:45,Sub2,File_158636,2015-01,PRODUCT_10712-PRODUCT_634,EMAN,4.0
148411,0.0,3.474203,4.782859,4.133455,5.576235,6.294996,8.250964,8.677573,9.775413,11.886012,...,200.916166,200.538143,201.633955,2015-01-01 00:11:20,Sub3,File_158740,2015-01,PRODUCT_10712-PRODUCT_634,EMAN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155550,0.0,2.116518,3.671717,4.800011,5.039060,6.500389,7.178479,9.405391,10.468659,10.895622,...,199.494443,200.977430,201.827405,2015-01-31 23:54:52,Sub2,File_166432,2015-01,PRODUCT_11471-PRODUCT_1386,,0
155679,0.0,3.222577,4.616407,5.661206,5.896912,6.445603,8.811819,9.329645,10.933297,10.326901,...,199.314993,200.665062,202.909933,2015-01-31 23:55:43,Sub3,File_166562,2015-01,PRODUCT_11471-PRODUCT_1386,,0
155430,0.0,3.688003,3.510717,4.292475,6.161303,7.006789,7.596664,9.030638,10.280821,10.332523,...,200.450067,200.553911,202.609093,2015-01-31 23:56:38,Sub1,File_166309,2015-01,PRODUCT_11471-PRODUCT_1386,,0
155551,0.0,2.062460,3.123475,5.745830,6.976661,7.853966,7.644802,8.959590,9.540880,11.970295,...,200.389667,201.568287,202.274165,2015-01-31 23:57:32,Sub2,File_166438,2015-01,PRODUCT_11471-PRODUCT_1386,,0


In [62]:
df_pressure_2015_02 =  df_pressure_data[df_pressure_data['check_time'] == arr_time_unique[2]]
df_product_2015_02 = df[(df['check_out_time'] == arr_time_unique[2]) | (df['check_in_time'] == arr_time_unique[2])]
df_pressure_2015_02['product_id'] = '####'
df_pressure_2015_02['error_code'] = '####'
df_pressure_2015_02['error_quantity'] = '####'
for index in range(len(df_pressure_2015_02)) :
        df_product_2015_02_find = df_product_2015_02[(df_product_2015_02['out_time'] >= df_pressure_2015_02.iloc[index , 201]) & (df_product_2015_02['in_time'] <= df_pressure_2015_02.iloc[index , 201])]
        df_pressure_2015_02.iloc[index , 205] ='-'.join(list(df_product_2015_02_find['product_id'].values))
        error_code = [x for x in df_product_2015_02_find['code'] if str(x) != 'nan']
        df_pressure_2015_02.iloc[index , 206] ='-'.join(error_code)
        error_quantity = [x for x in df_product_2015_02_find['errors'] if str(x) != 'nan']
        total_quantity = sum(error_quantity)
        df_pressure_2015_02.iloc[index , 207] = total_quantity

df_pressure_2015_02

<ipython-input-62-e2596126b27b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_02['product_id'] = '####'
<ipython-input-62-e2596126b27b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_02['error_code'] = '####'
<ipython-input-62-e2596126b27b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,check_time,product_id,error_code,error_quantity
155716,0.0,2.375014,3.742920,5.811090,5.234271,7.442935,8.841066,8.361159,10.195384,11.712607,...,200.262323,201.971713,202.359631,2015-01-31 23:59:18,Sub1,File_166606,2015-02,PRODUCT_11471-PRODUCT_1386,,0
155826,0.0,3.268227,3.073795,5.046188,6.253563,7.524525,8.181107,9.660563,9.844977,11.081232,...,200.324617,201.861068,201.248213,2015-02-01 00:00:12,Sub2,File_166723,2015-02,PRODUCT_11471-PRODUCT_1386,,0
155900,0.0,3.021206,4.544137,4.762393,6.867614,7.272047,8.883091,8.950854,10.543588,11.817695,...,200.159404,200.835738,202.360017,2015-02-01 00:01:03,Sub3,File_166802,2015-02,PRODUCT_11471-PRODUCT_1386,,0
155717,0.0,2.696978,3.125422,4.660921,5.146766,6.047054,7.556945,8.078691,10.023370,11.335847,...,200.219259,200.838613,202.709515,2015-02-01 00:01:58,Sub1,File_166613,2015-02,PRODUCT_11471-PRODUCT_1386,,0
155827,0.0,3.268810,3.504887,4.753588,5.705883,6.868135,8.014338,8.700425,10.362206,10.982954,...,200.798356,200.806298,202.745443,2015-02-01 00:02:52,Sub2,File_166728,2015-02,PRODUCT_11471-PRODUCT_1386,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129402,0.0,2.183467,3.798138,4.943805,6.422175,7.359794,8.515053,9.067130,9.357315,11.662133,...,199.970981,200.959035,202.623045,2015-02-28 23:45:45,Sub4,File_138218,2015-02,PRODUCT_1936-PRODUCT_12089,,0
129347,0.0,2.473305,3.036070,4.338374,6.370128,7.160955,8.363146,8.426387,10.465017,10.961330,...,200.100713,200.597582,202.146013,2015-02-28 23:55:07,Sub3,File_138169,2015-02,PRODUCT_12089-PRODUCT_1944,,0
129406,0.0,2.695140,3.636063,4.704955,5.713330,6.922048,7.944028,9.363974,10.602474,10.352894,...,200.451614,200.816221,201.513042,2015-02-28 23:55:45,Sub4,File_138220,2015-02,PRODUCT_12089-PRODUCT_1944,,0
129362,0.0,3.307364,4.379816,5.989254,5.365792,6.133666,7.827325,9.035899,9.438698,11.676843,...,199.825811,200.897331,201.252965,2015-02-28 23:57:43,Sub3,File_138176,2015-02,PRODUCT_12089-PRODUCT_1944,,0


In [63]:
df_pressure_2015_03 =  df_pressure_data[df_pressure_data['check_time'] == arr_time_unique[3]]
df_product_2015_03 = df[(df['check_out_time'] == arr_time_unique[3]) | (df['check_in_time'] == arr_time_unique[3])]
df_pressure_2015_03['product_id'] = '####'
df_pressure_2015_03['error_code'] = '####'
df_pressure_2015_03['error_quantity'] = '####'
for index in range(len(df_pressure_2015_03)) :
        df_product_2015_03_find = df_product_2015_03[(df_product_2015_03['out_time'] >= df_pressure_2015_03.iloc[index , 201]) & (df_product_2015_03['in_time'] <= df_pressure_2015_03.iloc[index , 201])]
        df_pressure_2015_03.iloc[index , 205] ='-'.join(list(df_product_2015_03_find['product_id'].values)) 
        error_code = [x for x in df_product_2015_03_find['code'] if str(x) != 'nan']
        df_pressure_2015_03.iloc[index , 206] ='-'.join(error_code)
        error_quantity = [x for x in df_product_2015_03_find['errors'] if str(x) != 'nan']
        total_quantity = sum(error_quantity)
        df_pressure_2015_03.iloc[index , 207] = total_quantity

df_pressure_2015_03

<ipython-input-63-46d70831f796>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_03['product_id'] = '####'
<ipython-input-63-46d70831f796>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_03['error_code'] = '####'
<ipython-input-63-46d70831f796>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,check_time,product_id,error_code,error_quantity
129408,0.0,3.632382,3.081330,4.307101,6.532208,6.159449,7.370004,8.272498,10.869978,11.644711,...,200.408158,200.411406,201.379797,2015-02-28 23:59:50,Sub1,File_138230,2015-03,PRODUCT_12089-PRODUCT_1944,,0
129646,0.0,3.715536,3.128528,4.972548,5.492673,7.309696,8.116509,8.844234,9.549928,11.730347,...,199.873688,201.891369,202.338011,2015-03-01 00:00:20,Sub3,File_138469,2015-03,PRODUCT_12089-PRODUCT_1944,,0
129759,0.0,3.262328,3.715904,5.229722,6.447847,6.095963,7.752543,9.347325,10.540830,10.106120,...,200.439319,201.702944,202.435719,2015-03-01 00:01:09,Sub4,File_138592,2015-03,PRODUCT_12089-PRODUCT_1944,,0
129647,0.0,3.230680,4.813394,4.743696,5.615920,7.977540,7.296432,8.593451,10.966986,10.250242,...,200.753467,201.202098,202.676515,2015-03-01 00:02:56,Sub3,File_138475,2015-03,PRODUCT_12089-PRODUCT_1944,,0
129760,0.0,3.525432,4.689690,4.038278,5.177359,6.868543,7.269913,9.669245,9.696001,10.332936,...,200.762523,200.864466,202.272910,2015-03-01 00:03:45,Sub4,File_138598,2015-03,PRODUCT_12089-PRODUCT_1944,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131736,0.0,2.560088,4.965893,4.982430,5.666144,6.003539,8.109713,9.973107,9.834427,10.907987,...,199.318390,200.086237,201.155134,2015-03-31 23:17:10,Sub4,File_140754,2015-03,PRODUCT_12826-PRODUCT_2804,,0
131750,0.0,3.540941,3.151896,4.575310,5.663312,6.908712,8.438335,8.855551,9.019524,11.136923,...,200.356755,201.181814,202.923394,2015-03-31 23:19:45,Sub4,File_140760,2015-03,PRODUCT_12826-PRODUCT_2804,,0
131751,0.0,3.058682,4.269020,5.158089,5.340299,6.216387,7.601015,8.045981,9.522198,10.318840,...,199.347646,200.745854,202.218628,2015-03-31 23:22:19,Sub4,File_140798,2015-03,PRODUCT_12826-PRODUCT_2804,,0
131752,0.0,3.820693,4.056977,5.355947,5.889925,7.695251,7.247822,9.266630,10.409044,10.990809,...,199.369740,200.440962,201.774242,2015-03-31 23:24:54,Sub4,File_140805,2015-03,PRODUCT_12826-PRODUCT_2804,,0


In [64]:
df_pressure_2015_04 =  df_pressure_data[df_pressure_data['check_time'] == arr_time_unique[4]]
df_product_2015_04 = df[(df['check_out_time'] == arr_time_unique[4]) | (df['check_in_time'] == arr_time_unique[4])]
df_pressure_2015_04['product_id'] = '####'
df_pressure_2015_04['error_code'] = '####'
df_pressure_2015_04['error_quantity'] = '####'
for index in range(len(df_pressure_2015_04)) :
        df_product_2015_04_find = df_product_2015_04[(df_product_2015_04['out_time'] >= df_pressure_2015_04.iloc[index , 201]) & (df_product_2015_04['in_time'] <= df_pressure_2015_04.iloc[index , 201])]
        df_pressure_2015_04.iloc[index , 205] ='-'.join(list(df_product_2015_04_find['product_id'].values)) 
        error_code = [x for x in df_product_2015_04_find['code'] if str(x) != 'nan']
        df_pressure_2015_04.iloc[index , 206] ='-'.join(error_code)
        error_quantity = [x for x in df_product_2015_04_find['errors'] if str(x) != 'nan']
        total_quantity = sum(error_quantity)
        df_pressure_2015_04.iloc[index , 207] = total_quantity

df_pressure_2015_04

<ipython-input-64-89856ea1bf68>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_04['product_id'] = '####'
<ipython-input-64-89856ea1bf68>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_04['error_code'] = '####'
<ipython-input-64-89856ea1bf68>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,check_time,product_id,error_code,error_quantity
131845,0.0,3.577121,3.515490,5.966419,5.662878,6.040691,7.992137,9.614686,10.478274,11.903056,...,200.402726,201.591202,202.296081,2015-04-01 01:26:58,Sub4,File_140877,2015-04,PRODUCT_2804-PRODUCT_12837,,0
131846,0.0,3.578075,4.568541,4.218949,5.357496,6.163420,7.029311,8.038203,9.916545,10.252726,...,200.907686,200.411853,201.521251,2015-04-01 01:34:07,Sub4,File_140884,2015-04,PRODUCT_2804-PRODUCT_12837,,0
131929,0.0,3.317690,4.263219,4.221149,5.636119,6.910274,8.096744,8.992009,10.935650,10.589885,...,200.209141,200.573552,202.115443,2015-04-01 01:59:21,Sub4,File_140960,2015-04,PRODUCT_2804-PRODUCT_12829,,0
131930,0.0,3.476515,4.301081,5.364077,6.556429,7.651811,8.727471,8.283140,10.904456,11.282233,...,200.438046,200.106624,202.413509,2015-04-01 02:01:56,Sub4,File_140966,2015-04,PRODUCT_2804-PRODUCT_12829,,0
131931,0.0,3.010739,4.149304,4.223477,5.625987,6.322847,7.143906,8.638137,10.031806,10.650576,...,199.908324,200.636413,202.585019,2015-04-01 02:04:30,Sub4,File_140972,2015-04,PRODUCT_2804-PRODUCT_12829,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121588,0.0,3.807017,3.919523,4.074144,5.545744,6.436575,8.206604,8.306952,9.893545,10.252999,...,200.955326,201.964563,201.154037,2015-04-30 23:53:24,Sub4,File_129841,2015-04,PRODUCT_13471-PRODUCT_3553,FQCH,4.0
121493,0.0,2.104064,3.443114,4.481737,6.705336,6.882450,8.707461,9.267186,9.136959,10.159834,...,200.538803,200.539643,201.632495,2015-04-30 23:54:59,Sub1,File_129740,2015-04,PRODUCT_13471-PRODUCT_3553,FQCH,4.0
121589,0.0,3.930519,3.677023,4.704639,5.637441,6.882296,7.126422,9.906606,9.979171,10.636447,...,199.664545,200.199419,202.359262,2015-04-30 23:55:59,Sub4,File_129848,2015-04,PRODUCT_13471-PRODUCT_3553,FQCH,4.0
121506,0.0,2.440019,3.700549,5.741263,5.046982,6.107905,8.775159,8.052696,10.226726,11.445073,...,199.027287,201.399310,201.064299,2015-04-30 23:57:33,Sub1,File_129747,2015-04,PRODUCT_13471-PRODUCT_3553,FQCH,4.0


In [65]:
df_pressure_2015_05 =  df_pressure_data[df_pressure_data['check_time'] == arr_time_unique[5]]
df_product_2015_05 = df[(df['check_out_time'] == arr_time_unique[5]) | (df['check_in_time'] == arr_time_unique[5])]
df_pressure_2015_05['product_id'] = '####'
df_pressure_2015_05['error_code'] = '####'
df_pressure_2015_05['error_quantity'] = '####'
for index in range(len(df_pressure_2015_05)) :
        df_product_2015_05_find = df_product_2015_05[(df_product_2015_05['out_time'] >= df_pressure_2015_05.iloc[index , 201]) & (df_product_2015_05['in_time'] <= df_pressure_2015_05.iloc[index , 201])]
        df_pressure_2015_05.iloc[index , 205] ='-'.join(list(df_product_2015_05_find['product_id'].values)) 
        error_code = [x for x in df_product_2015_05_find['code'] if str(x) != 'nan']
        df_pressure_2015_05.iloc[index , 206] ='-'.join(error_code)
        error_quantity = [x for x in df_product_2015_05_find['errors'] if str(x) != 'nan']
        total_quantity = sum(error_quantity)
        df_pressure_2015_05.iloc[index , 207] = total_quantity

df_pressure_2015_05

<ipython-input-65-62f8624d34fb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_05['product_id'] = '####'
<ipython-input-65-62f8624d34fb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_05['error_code'] = '####'
<ipython-input-65-62f8624d34fb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,check_time,product_id,error_code,error_quantity
121600,0.0,2.405976,3.898673,4.784562,5.511876,6.266233,8.195986,9.223300,10.813501,11.388309,...,199.951166,200.264377,201.158404,2015-05-01 00:00:07,Sub1,File_129859,2015-05,PRODUCT_13471-PRODUCT_3553,FQCH,4.0
121741,0.0,2.241978,3.112241,4.935901,5.282869,6.426578,8.179502,8.828404,10.565730,10.131997,...,200.730868,201.441461,202.335808,2015-05-01 00:01:08,Sub4,File_130020,2015-05,PRODUCT_13471-PRODUCT_3553,FQCH,4.0
121601,0.0,3.318513,4.369870,5.736618,6.224216,6.449883,7.573068,9.554152,10.914600,10.739238,...,199.989936,200.780219,202.668323,2015-05-01 00:02:40,Sub1,File_129866,2015-05,PRODUCT_13471-PRODUCT_3553,FQCH,4.0
121742,0.0,3.086959,3.954199,4.680419,6.699188,7.493407,7.883309,9.032649,10.797771,10.750624,...,200.641133,201.720723,201.781041,2015-05-01 00:03:43,Sub4,File_130028,2015-05,PRODUCT_13471-PRODUCT_3553,FQCH,4.0
121617,0.0,3.988957,4.741490,4.972297,6.329512,6.502344,8.419115,9.731521,9.747801,10.368437,...,199.538221,201.531298,202.159827,2015-05-01 00:05:14,Sub1,File_129873,2015-05,PRODUCT_13471-PRODUCT_3553,FQCH,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124238,0.0,2.480204,3.401650,4.883951,5.894396,7.946412,8.772616,9.292649,10.703409,11.350072,...,199.256405,200.566851,201.604599,2015-05-31 23:41:17,Sub3,File_132740,2015-05,PRODUCT_14067-PRODUCT_4460,,0
124239,0.0,3.328480,4.967952,4.604342,6.513653,7.106393,7.746167,9.565925,10.470801,11.477367,...,200.547730,201.445867,202.851688,2015-05-31 23:43:57,Sub3,File_132745,2015-05,PRODUCT_14067-PRODUCT_4460,,0
124250,0.0,3.758214,4.462076,4.462992,6.870476,7.749099,7.034299,9.968359,10.385748,11.424356,...,200.468995,200.000705,202.629111,2015-05-31 23:46:32,Sub3,File_132750,2015-05,PRODUCT_14067-PRODUCT_4460,,0
124251,0.0,2.095292,4.674659,4.551148,6.530737,7.960020,8.218445,8.843217,9.360524,11.612081,...,199.609525,200.869211,202.048180,2015-05-31 23:49:07,Sub3,File_132756,2015-05,PRODUCT_14067-PRODUCT_4460,,0


In [66]:
df_pressure_2015_06 =  df_pressure_data[df_pressure_data['check_time'] == arr_time_unique[6]]
df_product_2015_06 = df[(df['check_out_time'] == arr_time_unique[6]) | (df['check_in_time'] == arr_time_unique[6])]
df_pressure_2015_06['product_id'] = '####'
df_pressure_2015_06['error_code'] = '####'
df_pressure_2015_06['error_quantity'] = '####'
for index in range(len(df_pressure_2015_06)) :
        df_product_2015_06_find = df_product_2015_06[(df_product_2015_06['out_time'] >= df_pressure_2015_06.iloc[index , 201]) & (df_product_2015_06['in_time'] <= df_pressure_2015_06.iloc[index , 201])]
        df_pressure_2015_06.iloc[index , 205] ='-'.join(list(df_product_2015_06_find['product_id'].values))
        error_code = [x for x in df_product_2015_06_find['code'] if str(x) != 'nan']
        df_pressure_2015_06.iloc[index , 206] ='-'.join(error_code)
        error_quantity = [x for x in df_product_2015_06_find['errors'] if str(x) != 'nan']
        total_quantity = sum(error_quantity)
        df_pressure_2015_06.iloc[index , 207] = total_quantity

df_pressure_2015_06

<ipython-input-66-3b66a9847980>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_06['product_id'] = '####'
<ipython-input-66-3b66a9847980>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_06['error_code'] = '####'
<ipython-input-66-3b66a9847980>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,check_time,product_id,error_code,error_quantity
124286,0.0,3.525806,4.361731,4.660843,5.835245,7.753622,7.520466,9.871762,9.735352,10.471540,...,199.071321,201.776812,202.101772,2015-06-01 00:09:34,Sub1,File_132800,2015-06,PRODUCT_4460-PRODUCT_14061,,0
124406,0.0,3.887808,4.072649,4.029028,6.214101,6.604813,8.961782,8.820659,9.504477,11.391422,...,199.484985,200.588690,202.874303,2015-06-01 00:10:25,Sub3,File_132926,2015-06,PRODUCT_4460-PRODUCT_14061,,0
124299,0.0,3.046680,3.501942,5.618602,5.126147,6.517210,7.454219,9.147404,10.487255,10.076681,...,200.803141,200.644809,201.386963,2015-06-01 00:12:07,Sub1,File_132805,2015-06,PRODUCT_4460-PRODUCT_14061,,0
124407,0.0,2.091460,3.457234,4.920597,5.816087,6.311335,7.432040,9.739847,9.013760,11.408823,...,199.944823,201.971825,201.513430,2015-06-01 00:13:03,Sub3,File_132933,2015-06,PRODUCT_4460-PRODUCT_14061,,0
124300,0.0,3.908076,3.390046,5.193727,6.578549,6.502106,7.725726,8.034305,10.024297,11.853867,...,200.236174,200.135727,202.292494,2015-06-01 00:14:41,Sub1,File_132810,2015-06,PRODUCT_4460-PRODUCT_14061,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120619,0.0,2.352523,4.741621,4.834002,5.093115,7.664855,7.358725,9.617148,10.252391,10.064671,...,200.132063,201.321894,201.420133,2015-06-30 23:52:57,Sub4,File_128833,2015-06,PRODUCT_5324,,0
120470,0.0,3.405276,4.033666,5.284785,5.461120,7.930150,7.754129,8.907333,9.293569,11.770010,...,200.398206,200.549141,201.651459,2015-06-30 23:54:31,Sub1,File_128638,2015-06,PRODUCT_5324-PRODUCT_14648,,0
120661,0.0,2.200115,4.277467,4.940291,6.028627,6.431504,8.616124,9.556333,9.581568,11.291147,...,200.176596,201.609820,201.331679,2015-06-30 23:55:32,Sub4,File_128839,2015-06,PRODUCT_5324-PRODUCT_14648,,0
120471,0.0,2.049316,3.091126,4.057189,5.535628,6.795312,8.969535,9.145273,9.280483,11.227849,...,200.726879,200.671866,202.510974,2015-06-30 23:57:05,Sub1,File_128642,2015-06,PRODUCT_5324-PRODUCT_14648,,0


In [67]:
df_pressure_2015_07 =  df_pressure_data[df_pressure_data['check_time'] == arr_time_unique[7]]
df_product_2015_07 = df[(df['check_out_time'] == arr_time_unique[7]) | (df['check_in_time'] == arr_time_unique[7])]
df_pressure_2015_07['product_id'] = '####'
df_pressure_2015_07['error_code'] = '####'
df_pressure_2015_07['error_quantity'] = '####'
for index in range(len(df_pressure_2015_07)) :
        df_product_2015_07_find = df_product_2015_07[(df_product_2015_07['out_time'] >= df_pressure_2015_07.iloc[index , 201]) & (df_product_2015_07['in_time'] <= df_pressure_2015_07.iloc[index , 201])]
        df_pressure_2015_07.iloc[index , 205] ='-'.join(list(df_product_2015_07_find['product_id'].values)) 
        error_code = [x for x in df_product_2015_07_find['code'] if str(x) != 'nan']
        df_pressure_2015_07.iloc[index , 206] ='-'.join(error_code)
        error_quantity = [x for x in df_product_2015_07_find['errors'] if str(x) != 'nan']
        total_quantity = sum(error_quantity)
        df_pressure_2015_07.iloc[index , 207] = total_quantity

df_pressure_2015_07

<ipython-input-67-267fc9902230>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_07['product_id'] = '####'
<ipython-input-67-267fc9902230>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pressure_2015_07['error_code'] = '####'
<ipython-input-67-267fc9902230>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,check_time,product_id,error_code,error_quantity
120678,0.0,3.685674,3.509761,5.991408,5.962210,6.539211,7.822558,9.612516,10.778840,10.843096,...,200.845077,200.466627,201.601451,2015-07-01 00:13:03,Sub1,File_128858,2015-07,PRODUCT_14648-PRODUCT_5331,,0
120778,0.0,2.577260,3.892418,4.571750,6.261464,7.189844,8.076296,8.814575,9.949080,11.505214,...,200.294007,201.638715,201.269383,2015-07-01 00:13:59,Sub4,File_128971,2015-07,PRODUCT_14648-PRODUCT_5331,,0
120679,0.0,2.299570,3.614869,5.931267,6.135602,6.724476,7.586307,9.840393,10.449074,11.510568,...,200.681395,200.955351,202.758543,2015-07-01 00:15:37,Sub1,File_128866,2015-07,PRODUCT_14648-PRODUCT_5331,,0
120779,0.0,3.547338,4.792247,4.098948,5.190952,6.665897,8.199030,9.050905,9.335861,10.287121,...,199.907914,200.934504,202.653632,2015-07-01 00:16:34,Sub4,File_128977,2015-07,PRODUCT_14648-PRODUCT_5331,,0
120680,0.0,2.890058,4.510281,4.564879,6.623905,7.734130,8.204949,9.024092,10.679860,11.388066,...,199.790501,200.258699,202.734349,2015-07-01 00:18:12,Sub1,File_128871,2015-07,PRODUCT_14648-PRODUCT_5331,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186755,0.0,2.931815,3.039504,5.962543,6.046933,6.575745,7.143761,8.365927,9.133804,10.985532,...,199.756017,201.094636,201.726737,2015-07-11 18:03:50,Sub2,File_199504,2015-07,PRODUCT_14849-PRODUCT_5604,,0
186770,0.0,3.630130,4.978932,4.390446,5.711679,6.356601,8.754259,9.606328,9.658834,11.409551,...,199.810128,201.865581,202.114220,2015-07-11 18:04:28,Sub3,File_199518,2015-07,PRODUCT_14849-PRODUCT_5604,,0
186787,0.0,2.394231,3.119259,5.783146,6.586384,6.113767,7.774616,8.288652,10.241900,11.189193,...,200.994531,201.110963,201.023481,2015-07-11 18:05:47,Sub4,File_199530,2015-07,PRODUCT_14849-PRODUCT_5604,,0
186754,0.0,3.855237,3.817676,5.714153,5.665591,7.170642,7.624416,8.157029,10.176965,10.818476,...,199.286844,200.238973,202.587612,2015-07-11 18:06:25,Sub1,File_199499,2015-07,PRODUCT_14849-PRODUCT_5604,,0


In [68]:
df_result = pd.concat([df_pressure_2014_12 ,df_pressure_2015_01,df_pressure_2015_02,df_pressure_2015_03,df_pressure_2015_04,df_pressure_2015_05,df_pressure_2015_06,df_pressure_2015_07])
df_result.drop(['check_time'] , axis=1 , inplace=True)
print(len(df_result))
df_result

198224


,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_198,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,product_id,error_code,error_quantity
0,0.0,2.639808,4.771886,5.979325,6.682468,7.285566,7.206227,8.655227,10.800956,10.964720,...,199.199552,199.762799,201.182017,201.180853,2014-12-10 00:09:50,Sub1,File_1,PRODUCT_141-PRODUCT_10165,BPAK,5.0
1,0.0,2.795488,3.251834,4.499492,5.502105,6.449215,8.087633,9.946180,10.219085,10.743145,...,199.259451,199.469542,201.953278,201.091001,2014-12-10 00:15:28,Sub1,File_2,PRODUCT_141-PRODUCT_10165,BPAK,5.0
8,0.0,2.989949,3.731346,5.592048,6.855218,7.751051,7.899846,8.706237,9.754890,11.872049,...,199.712650,199.619340,200.246648,202.049603,2014-12-10 00:16:05,Sub2,File_9,PRODUCT_141-PRODUCT_10165,BPAK,5.0
2,0.0,3.807030,3.503354,4.655089,6.302633,7.726263,8.249255,9.675065,9.591786,10.676826,...,198.747466,199.035431,200.608281,201.498844,2014-12-10 00:20:41,Sub1,File_3,PRODUCT_141-PRODUCT_10165,BPAK,5.0
9,0.0,3.740250,4.023481,5.676871,6.898762,6.879299,8.869699,9.950092,9.061035,11.285521,...,198.990163,200.082805,201.440099,201.270897,2014-12-10 00:21:42,Sub2,File_10,PRODUCT_141-PRODUCT_10165,BPAK,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186755,0.0,2.931815,3.039504,5.962543,6.046933,6.575745,7.143761,8.365927,9.133804,10.985532,...,198.944525,199.756017,201.094636,201.726737,2015-07-11 18:03:50,Sub2,File_199504,PRODUCT_14849-PRODUCT_5604,,0
186770,0.0,3.630130,4.978932,4.390446,5.711679,6.356601,8.754259,9.606328,9.658834,11.409551,...,198.997033,199.810128,201.865581,202.114220,2015-07-11 18:04:28,Sub3,File_199518,PRODUCT_14849-PRODUCT_5604,,0
186787,0.0,2.394231,3.119259,5.783146,6.586384,6.113767,7.774616,8.288652,10.241900,11.189193,...,199.458638,200.994531,201.110963,201.023481,2015-07-11 18:05:47,Sub4,File_199530,PRODUCT_14849-PRODUCT_5604,,0
186754,0.0,3.855237,3.817676,5.714153,5.665591,7.170642,7.624416,8.157029,10.176965,10.818476,...,198.456641,199.286844,200.238973,202.587612,2015-07-11 18:06:25,Sub1,File_199499,PRODUCT_14849-PRODUCT_5604,,0


In [14]:
def check_error(x):
    if x > 96.0:
        return 1
    else:
        return 0
df_result['isError'] = df_result['error_quantity'].apply(check_error)
df_result

,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5,pressure_6,pressure_7,pressure_8,pressure_9,pressure_10,...,pressure_199,pressure_200,pressure_201,start_at,sub,file_name,product_id,error_code,error_quantity,isError
0,0.0,2.639808,4.771886,5.979325,6.682468,7.285566,7.206227,8.655227,10.800956,10.964720,...,199.762799,201.182017,201.180853,2014-12-10 00:09:50,Sub1,File_1,PRODUCT_141-PRODUCT_10165,BPAK,5.0,0
1,0.0,2.795488,3.251834,4.499492,5.502105,6.449215,8.087633,9.946180,10.219085,10.743145,...,199.469542,201.953278,201.091001,2014-12-10 00:15:28,Sub1,File_2,PRODUCT_141-PRODUCT_10165,BPAK,5.0,0
8,0.0,2.989949,3.731346,5.592048,6.855218,7.751051,7.899846,8.706237,9.754890,11.872049,...,199.619340,200.246648,202.049603,2014-12-10 00:16:05,Sub2,File_9,PRODUCT_141-PRODUCT_10165,BPAK,5.0,0
2,0.0,3.807030,3.503354,4.655089,6.302633,7.726263,8.249255,9.675065,9.591786,10.676826,...,199.035431,200.608281,201.498844,2014-12-10 00:20:41,Sub1,File_3,PRODUCT_141-PRODUCT_10165,BPAK,5.0,0
9,0.0,3.740250,4.023481,5.676871,6.898762,6.879299,8.869699,9.950092,9.061035,11.285521,...,200.082805,201.440099,201.270897,2014-12-10 00:21:42,Sub2,File_10,PRODUCT_141-PRODUCT_10165,BPAK,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186755,0.0,2.931815,3.039504,5.962543,6.046933,6.575745,7.143761,8.365927,9.133804,10.985532,...,199.756017,201.094636,201.726737,2015-07-11 18:03:50,Sub2,File_199504,PRODUCT_14849-PRODUCT_5604,,0,0
186770,0.0,3.630130,4.978932,4.390446,5.711679,6.356601,8.754259,9.606328,9.658834,11.409551,...,199.810128,201.865581,202.114220,2015-07-11 18:04:28,Sub3,File_199518,PRODUCT_14849-PRODUCT_5604,,0,0
186787,0.0,2.394231,3.119259,5.783146,6.586384,6.113767,7.774616,8.288652,10.241900,11.189193,...,200.994531,201.110963,201.023481,2015-07-11 18:05:47,Sub4,File_199530,PRODUCT_14849-PRODUCT_5604,,0,0
186754,0.0,3.855237,3.817676,5.714153,5.665591,7.170642,7.624416,8.157029,10.176965,10.818476,...,199.286844,200.238973,202.587612,2015-07-11 18:06:25,Sub1,File_199499,PRODUCT_14849-PRODUCT_5604,,0,0


In [118]:
df_result_detail = pd.DataFrame()
df_result_1 = df_result.iloc[0 : 10000]
for i in range(len(df_result_1)):
    array_productId = df_result_1.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_1.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail=df_result_detail.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail

,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_10165,2014-12-10 00:09:50,Sub1
1,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_141,2014-12-10 00:09:50,Sub1
2,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_10165,2014-12-10 00:15:28,Sub1
3,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_141,2014-12-10 00:15:28,Sub1
4,BPAK,5.0,File_9,0.0,0.0,11.872049,101.239317,102.399545,103.670321,104.667630,...,93.132023,95.985287,95.470922,96.806140,97.468899,98.462504,99.112021,PRODUCT_10165,2014-12-10 00:16:05,Sub2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20347,FQCH,3.0,File_71124,0.0,0.0,10.969140,101.383964,101.510524,102.673023,103.683831,...,94.530593,94.863310,95.565409,97.555637,98.516587,99.472681,99.693941,PRODUCT_10421,2014-12-20 12:57:22,Sub4
20348,FQCH,3.0,File_70933,0.0,0.0,10.593495,100.937048,101.695259,102.277439,103.829709,...,93.842611,95.776537,95.132464,97.831518,97.251293,98.177016,99.861525,PRODUCT_377,2014-12-20 12:58:45,Sub2
20349,FQCH,3.0,File_70933,0.0,0.0,10.593495,100.937048,101.695259,102.277439,103.829709,...,93.842611,95.776537,95.132464,97.831518,97.251293,98.177016,99.861525,PRODUCT_10421,2014-12-20 12:58:45,Sub2
20350,FQCH,3.0,File_71286,0.0,0.0,10.506920,100.496522,102.089559,102.669640,104.174599,...,93.334100,94.505313,96.016065,97.166737,98.626749,98.489116,100.788562,PRODUCT_377,2014-12-20 12:59:44,Sub3


In [120]:
# df_result_detail.to_csv('result-final.csv')
df_result_2 = df_result.iloc[10000 : 20000]
for i in range(len(df_result_2)):
    array_productId = df_result_2.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_2.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail=df_result_detail.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail

,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_10165,2014-12-10 00:09:50,Sub1
1,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_141,2014-12-10 00:09:50,Sub1
2,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_10165,2014-12-10 00:15:28,Sub1
3,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_141,2014-12-10 00:15:28,Sub1
4,BPAK,5.0,File_9,0.0,0.0,11.872049,101.239317,102.399545,103.670321,104.667630,...,93.132023,95.985287,95.470922,96.806140,97.468899,98.462504,99.112021,PRODUCT_10165,2014-12-10 00:16:05,Sub2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40389,,0.0,File_142852,0.0,0.0,10.886232,101.715523,101.456367,102.034781,103.390002,...,94.308393,94.560711,95.605290,96.807848,98.826021,98.816118,100.171500,PRODUCT_589,2014-12-30 01:14:09,Sub4
40390,,0.0,File_142557,0.0,0.0,10.411234,101.647673,102.305212,102.903744,104.609566,...,93.187697,94.361855,95.414447,96.284348,97.127011,98.180086,99.523523,PRODUCT_10669,2014-12-30 01:15:08,Sub1
40391,,0.0,File_142557,0.0,0.0,10.411234,101.647673,102.305212,102.903744,104.609566,...,93.187697,94.361855,95.414447,96.284348,97.127011,98.180086,99.523523,PRODUCT_589,2014-12-30 01:15:08,Sub1
40392,,0.0,File_142675,0.0,0.0,11.511390,101.748770,102.864176,103.975097,103.525926,...,94.477783,94.280432,95.468175,97.400731,98.323680,99.945915,100.827615,PRODUCT_10669,2014-12-30 01:16:08,Sub2


In [122]:
df_result_3 = df_result.iloc[20000 : 30000]
for i in range(len(df_result_3)):
    array_productId = df_result_3.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_3.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail=df_result_detail.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail

,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_10165,2014-12-10 00:09:50,Sub1
1,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_141,2014-12-10 00:09:50,Sub1
2,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_10165,2014-12-10 00:15:28,Sub1
3,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_141,2014-12-10 00:15:28,Sub1
4,BPAK,5.0,File_9,0.0,0.0,11.872049,101.239317,102.399545,103.670321,104.667630,...,93.132023,95.985287,95.470922,96.806140,97.468899,98.462504,99.112021,PRODUCT_10165,2014-12-10 00:16:05,Sub2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60874,,0.0,File_206532,0.0,0.0,10.130683,100.823063,102.523042,102.313943,104.059565,...,93.060751,94.802728,95.316084,97.347583,97.692867,98.493958,99.682631,PRODUCT_823,2015-01-08 10:21:05,Sub2
60875,,0.0,File_206559,0.0,0.0,11.501930,101.700087,101.304504,103.051960,104.283460,...,94.607395,94.646937,95.278317,97.147743,97.124542,98.136323,100.709055,PRODUCT_10891,2015-01-08 10:22:04,Sub3
60876,,0.0,File_206559,0.0,0.0,11.501930,101.700087,101.304504,103.051960,104.283460,...,94.607395,94.646937,95.278317,97.147743,97.124542,98.136323,100.709055,PRODUCT_823,2015-01-08 10:22:04,Sub3
60877,,0.0,File_206583,0.0,0.0,11.276412,100.594468,102.841891,102.140011,103.409229,...,93.130838,95.409373,96.703593,97.423121,98.505357,98.799314,99.142894,PRODUCT_10891,2015-01-08 10:22:59,Sub4


In [125]:
df_result_4= df_result.iloc[30000 : 40000]
for i in range(len(df_result_4)):
    array_productId = df_result_4.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_4.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail=df_result_detail.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail

,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_10165,2014-12-10 00:09:50,Sub1
1,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_141,2014-12-10 00:09:50,Sub1
2,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_10165,2014-12-10 00:15:28,Sub1
3,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_141,2014-12-10 00:15:28,Sub1
4,BPAK,5.0,File_9,0.0,0.0,11.872049,101.239317,102.399545,103.670321,104.667630,...,93.132023,95.985287,95.470922,96.806140,97.468899,98.462504,99.112021,PRODUCT_10165,2014-12-10 00:16:05,Sub2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81444,,0.0,File_67425,0.0,0.0,10.918501,100.634930,101.397127,103.333139,103.260746,...,94.442396,94.543658,95.871471,96.509741,97.205914,98.247136,100.218773,PRODUCT_11164,2015-01-19 07:50:09,Sub1
81445,,0.0,File_67487,0.0,0.0,11.495506,101.531928,101.171156,102.043344,104.304928,...,93.796466,94.660964,95.456534,97.772043,98.485124,98.228425,99.810534,PRODUCT_1053,2015-01-19 07:51:02,Sub2
81446,,0.0,File_67487,0.0,0.0,11.495506,101.531928,101.171156,102.043344,104.304928,...,93.796466,94.660964,95.456534,97.772043,98.485124,98.228425,99.810534,PRODUCT_11164,2015-01-19 07:51:02,Sub2
81447,,0.0,File_67569,0.0,0.0,10.433290,100.681971,101.434519,102.157382,103.330961,...,94.926004,94.744526,95.730276,96.473727,98.140748,99.939938,100.646112,PRODUCT_1053,2015-01-19 07:51:53,Sub3


In [128]:
df_result_5= df_result.iloc[40000 : 50000]
for i in range(len(df_result_5)):
    array_productId = df_result_5.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_5.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail=df_result_detail.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail

,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_10165,2014-12-10 00:09:50,Sub1
1,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_141,2014-12-10 00:09:50,Sub1
2,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_10165,2014-12-10 00:15:28,Sub1
3,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_141,2014-12-10 00:15:28,Sub1
4,BPAK,5.0,File_9,0.0,0.0,11.872049,101.239317,102.399545,103.670321,104.667630,...,93.132023,95.985287,95.470922,96.806140,97.468899,98.462504,99.112021,PRODUCT_10165,2014-12-10 00:16:05,Sub2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101522,,0.0,File_140060,0.0,0.0,10.209474,100.173245,102.337481,103.780195,104.228520,...,93.241622,95.498316,96.361997,96.528192,98.443659,99.584925,100.654552,PRODUCT_1279,2015-01-28 09:11:56,Sub3
101523,,0.0,File_140198,0.0,0.0,11.854924,100.191205,102.418478,103.472986,104.233077,...,94.953652,95.781307,96.085060,96.364092,98.373588,99.044468,100.125997,PRODUCT_11397,2015-01-28 09:12:46,Sub4
101524,,0.0,File_140198,0.0,0.0,11.854924,100.191205,102.418478,103.472986,104.233077,...,94.953652,95.781307,96.085060,96.364092,98.373588,99.044468,100.125997,PRODUCT_1279,2015-01-28 09:12:46,Sub4
101525,,0.0,File_139923,0.0,0.0,11.838561,101.839760,102.072921,102.886431,104.877321,...,93.445614,95.611619,95.297812,97.545720,97.216650,99.551388,100.396581,PRODUCT_11397,2015-01-28 09:13:42,Sub2


In [18]:
# df_result_detail = pd.read_csv('result-final.csv')
# df_result_detail
df_result_6= df_result.iloc[50000 : 60000]
for i in range(len(df_result_6)):
    array_productId = df_result_6.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_6.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail=df_result_detail.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail

,Unnamed: 0,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,0.0,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_10165,2014-12-10 00:09:50,Sub1
1,1.0,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,...,93.880783,94.621552,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_141,2014-12-10 00:09:50,Sub1
2,2.0,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_10165,2014-12-10 00:15:28,Sub1
3,3.0,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,...,94.714893,95.050203,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_141,2014-12-10 00:15:28,Sub1
4,4.0,BPAK,5.0,File_9,0.0,0.0,11.872049,101.239317,102.399545,103.670321,...,93.132023,95.985287,95.470922,96.806140,97.468899,98.462504,99.112021,PRODUCT_10165,2014-12-10 00:16:05,Sub2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121989,NaN,BLOK-BPAK-EMAN,44.0,File_205455,0.0,0.0,10.022323,101.955816,101.161301,102.385698,...,94.114934,94.346857,96.412928,97.684066,98.485202,98.312645,99.779788,PRODUCT_1553,2015-02-07 08:44:29,Sub3
121990,NaN,BLOK-BPAK-EMAN,44.0,File_205455,0.0,0.0,10.022323,101.955816,101.161301,102.385698,...,94.114934,94.346857,96.412928,97.684066,98.485202,98.312645,99.779788,PRODUCT_11621,2015-02-07 08:44:29,Sub3
121991,NaN,BLOK-BPAK-EMAN,44.0,File_205378,0.0,0.0,11.709261,101.249325,101.174798,103.878921,...,94.888642,94.417991,95.093497,97.566631,97.212657,98.928270,99.142610,PRODUCT_1553,2015-02-07 08:46:03,Sub2
121992,NaN,BLOK-BPAK-EMAN,44.0,File_205378,0.0,0.0,11.709261,101.249325,101.174798,103.878921,...,94.888642,94.417991,95.093497,97.566631,97.212657,98.928270,99.142610,PRODUCT_1553,2015-02-07 08:46:03,Sub2


In [24]:
df_result_detail_2 = pd.DataFrame()
df_result_7= df_result.iloc[60000 : 70000]
for i in range(len(df_result_7)):
    array_productId = df_result_7.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_7.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_2=df_result_detail_2.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_2

,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,BLOK-BPAK-EMAN,44.0,File_205458,0.0,0.0,11.514816,101.835891,101.539465,102.567435,103.739339,...,94.587659,94.567113,95.403375,96.869982,98.175242,99.791582,100.386716,PRODUCT_1553,2015-02-07 08:47:00,Sub3
1,BLOK-BPAK-EMAN,44.0,File_205458,0.0,0.0,11.514816,101.835891,101.539465,102.567435,103.739339,...,94.587659,94.567113,95.403375,96.869982,98.175242,99.791582,100.386716,PRODUCT_1553,2015-02-07 08:47:00,Sub3
2,BLOK-BPAK-EMAN,44.0,File_205458,0.0,0.0,11.514816,101.835891,101.539465,102.567435,103.739339,...,94.587659,94.567113,95.403375,96.869982,98.175242,99.791582,100.386716,PRODUCT_11621,2015-02-07 08:47:00,Sub3
3,BLOK-BPAK-EMAN,44.0,File_205380,0.0,0.0,11.647743,101.810800,102.242753,102.093226,104.201849,...,94.309919,95.224939,95.743453,97.504714,97.392654,99.346373,99.474873,PRODUCT_1553,2015-02-07 08:48:37,Sub2
4,BLOK-BPAK-EMAN,44.0,File_205380,0.0,0.0,11.647743,101.810800,102.242753,102.093226,104.201849,...,94.309919,95.224939,95.743453,97.504714,97.392654,99.346373,99.474873,PRODUCT_1553,2015-02-07 08:48:37,Sub2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19828,BLOK,1.0,File_60617,0.0,0.0,11.336917,101.139545,101.420358,103.035112,103.885193,...,93.530207,95.797239,96.509296,96.498990,97.130803,99.352031,100.850616,PRODUCT_11838,2015-02-18 00:56:00,Sub3
19829,BLOK,1.0,File_60336,0.0,0.0,10.342799,101.189656,101.033792,102.910190,103.064572,...,93.834291,95.585631,95.006779,97.747028,98.844140,99.405231,100.627787,PRODUCT_1730,2015-02-18 00:57:00,Sub1
19830,BLOK,1.0,File_60336,0.0,0.0,10.342799,101.189656,101.033792,102.910190,103.064572,...,93.834291,95.585631,95.006779,97.747028,98.844140,99.405231,100.627787,PRODUCT_11838,2015-02-18 00:57:00,Sub1
19831,BLOK,1.0,File_60475,0.0,0.0,11.265759,101.693715,101.095988,103.232462,104.670883,...,93.575745,95.169431,96.400040,96.095134,98.177284,98.754697,99.718982,PRODUCT_1730,2015-02-18 00:57:55,Sub2


In [30]:
# df_result_detail_2.to_csv('result-final-2.csv')
df_result_8= df_result.iloc[70000 : 80000]
for i in range(len(df_result_8)):
    array_productId = df_result_8.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_8.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_2=df_result_detail_2.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_2

,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,BLOK-BPAK-EMAN,44.0,File_205458,0.0,0.0,11.514816,101.835891,101.539465,102.567435,103.739339,...,94.587659,94.567113,95.403375,96.869982,98.175242,99.791582,100.386716,PRODUCT_1553,2015-02-07 08:47:00,Sub3
1,BLOK-BPAK-EMAN,44.0,File_205458,0.0,0.0,11.514816,101.835891,101.539465,102.567435,103.739339,...,94.587659,94.567113,95.403375,96.869982,98.175242,99.791582,100.386716,PRODUCT_1553,2015-02-07 08:47:00,Sub3
2,BLOK-BPAK-EMAN,44.0,File_205458,0.0,0.0,11.514816,101.835891,101.539465,102.567435,103.739339,...,94.587659,94.567113,95.403375,96.869982,98.175242,99.791582,100.386716,PRODUCT_11621,2015-02-07 08:47:00,Sub3
3,BLOK-BPAK-EMAN,44.0,File_205380,0.0,0.0,11.647743,101.810800,102.242753,102.093226,104.201849,...,94.309919,95.224939,95.743453,97.504714,97.392654,99.346373,99.474873,PRODUCT_1553,2015-02-07 08:48:37,Sub2
4,BLOK-BPAK-EMAN,44.0,File_205380,0.0,0.0,11.647743,101.810800,102.242753,102.093226,104.201849,...,94.309919,95.224939,95.743453,97.504714,97.392654,99.346373,99.474873,PRODUCT_1553,2015-02-07 08:48:37,Sub2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40753,,0.0,File_132968,0.0,0.0,10.688693,100.753097,101.855408,102.087100,103.861054,...,93.973373,95.793191,95.728033,96.779579,97.188703,99.849850,100.989994,PRODUCT_12058,2015-02-28 01:19:03,Sub3
40754,,0.0,File_132809,0.0,0.0,11.339003,101.446499,102.781411,103.149937,103.452520,...,94.041667,94.237741,96.398160,96.842013,98.362720,98.167399,99.720750,PRODUCT_1918,2015-02-28 01:20:34,Sub1
40755,,0.0,File_132809,0.0,0.0,11.339003,101.446499,102.781411,103.149937,103.452520,...,94.041667,94.237741,96.398160,96.842013,98.362720,98.167399,99.720750,PRODUCT_12058,2015-02-28 01:20:34,Sub1
40756,,0.0,File_132972,0.0,0.0,10.650314,100.362602,102.963398,102.746919,104.650384,...,93.434769,94.653008,96.641681,96.286178,97.257454,99.386875,99.818121,PRODUCT_1918,2015-02-28 01:21:40,Sub3


In [33]:
df_result_9= df_result.iloc[80000 : 90000]
for i in range(len(df_result_9)):
    array_productId = df_result_9.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_9.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_2=df_result_detail_2.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_2

,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,BLOK-BPAK-EMAN,44.0,File_205458,0.0,0.0,11.514816,101.835891,101.539465,102.567435,103.739339,...,94.587659,94.567113,95.403375,96.869982,98.175242,99.791582,100.386716,PRODUCT_1553,2015-02-07 08:47:00,Sub3
1,BLOK-BPAK-EMAN,44.0,File_205458,0.0,0.0,11.514816,101.835891,101.539465,102.567435,103.739339,...,94.587659,94.567113,95.403375,96.869982,98.175242,99.791582,100.386716,PRODUCT_1553,2015-02-07 08:47:00,Sub3
2,BLOK-BPAK-EMAN,44.0,File_205458,0.0,0.0,11.514816,101.835891,101.539465,102.567435,103.739339,...,94.587659,94.567113,95.403375,96.869982,98.175242,99.791582,100.386716,PRODUCT_11621,2015-02-07 08:47:00,Sub3
3,BLOK-BPAK-EMAN,44.0,File_205380,0.0,0.0,11.647743,101.810800,102.242753,102.093226,104.201849,...,94.309919,95.224939,95.743453,97.504714,97.392654,99.346373,99.474873,PRODUCT_1553,2015-02-07 08:48:37,Sub2
4,BLOK-BPAK-EMAN,44.0,File_205380,0.0,0.0,11.647743,101.810800,102.242753,102.093226,104.201849,...,94.309919,95.224939,95.743453,97.504714,97.392654,99.346373,99.474873,PRODUCT_1553,2015-02-07 08:48:37,Sub2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61396,,0.0,File_202747,0.0,0.0,11.025844,100.452614,102.956353,102.340505,104.202844,...,94.670484,94.472094,95.259000,96.727723,97.544537,99.398855,99.998036,PRODUCT_2232,2015-03-10 11:02:34,Sub3
61397,,0.0,File_202803,0.0,0.0,11.119439,101.161292,102.358033,102.269222,103.763505,...,93.563831,95.068122,95.589395,97.873772,98.319369,98.302094,99.796621,PRODUCT_12294,2015-03-10 11:03:23,Sub4
61398,,0.0,File_202803,0.0,0.0,11.119439,101.161292,102.358033,102.269222,103.763505,...,93.563831,95.068122,95.589395,97.873772,98.319369,98.302094,99.796621,PRODUCT_2232,2015-03-10 11:03:23,Sub4
61399,,0.0,File_202712,0.0,0.0,11.328417,101.485115,101.634733,102.747052,104.687645,...,93.857739,95.125950,96.457665,96.112283,98.437030,99.193421,99.846950,PRODUCT_12294,2015-03-10 11:04:25,Sub2


In [72]:
df_result_detail_3 = pd.DataFrame()
df_result_12= df_result.iloc[110000 : 120000]
for i in range(len(df_result_12)):
    array_productId = df_result_12.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_12.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_3=df_result_detail_3.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_3

,error_code,error_quantity,file_name,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,pressure_104,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_2775,2015-03-30 14:13:35,Sub4
1,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_12788,2015-03-30 14:13:35,Sub4
2,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_2775,2015-03-30 14:15:16,Sub3
3,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_12788,2015-03-30 14:15:16,Sub3
4,,0.0,File_134807,0.0,11.434745,101.389907,102.658200,102.258518,103.709444,105.403732,...,94.969882,94.204441,96.452536,96.909193,98.588181,98.908564,100.870959,PRODUCT_2775,2015-03-30 14:16:09,Sub4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20633,,0.0,File_4038,0.0,11.381717,100.106197,102.150038,103.075747,103.012920,104.348350,...,94.685223,94.822915,95.780384,96.917454,98.913041,98.000818,99.838253,PRODUCT_13045,2015-04-11 13:36:00,Sub1
20634,,0.0,File_4197,0.0,11.129925,101.287698,101.322953,102.617015,104.379498,105.248279,...,94.123648,94.895232,95.188753,97.311726,97.044170,99.569851,100.302927,PRODUCT_3079,2015-04-11 13:36:51,Sub3
20635,,0.0,File_4197,0.0,11.129925,101.287698,101.322953,102.617015,104.379498,105.248279,...,94.123648,94.895232,95.188753,97.311726,97.044170,99.569851,100.302927,PRODUCT_13045,2015-04-11 13:36:51,Sub3
20636,,0.0,File_4354,0.0,11.443522,100.406851,102.195784,102.707093,104.054804,104.988553,...,93.272497,94.126898,95.116116,97.588783,98.324682,98.297749,100.396569,PRODUCT_3079,2015-04-11 13:37:40,Sub4


In [75]:
df_result_13= df_result.iloc[120000 : 130000]
for i in range(len(df_result_13)):
    array_productId = df_result_13.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_13.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_3=df_result_detail_3.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_3

,error_code,error_quantity,file_name,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,pressure_104,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_2775,2015-03-30 14:13:35,Sub4
1,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_12788,2015-03-30 14:13:35,Sub4
2,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_2775,2015-03-30 14:15:16,Sub3
3,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_12788,2015-03-30 14:15:16,Sub3
4,,0.0,File_134807,0.0,11.434745,101.389907,102.658200,102.258518,103.709444,105.403732,...,94.969882,94.204441,96.452536,96.909193,98.588181,98.908564,100.870959,PRODUCT_2775,2015-03-30 14:16:09,Sub4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41169,,0.0,File_77422,0.0,11.835068,100.584339,101.505370,103.834275,104.639308,104.946663,...,94.386076,95.394655,96.717641,96.937981,98.576089,99.520511,100.205042,PRODUCT_3369,2015-04-21 20:06:12,Sub4
41170,,0.0,File_77235,0.0,10.558259,100.291312,102.168966,103.072892,104.204259,105.001762,...,93.446136,95.890642,95.166324,96.301512,97.018546,99.777512,100.352210,PRODUCT_13269,2015-04-21 20:07:44,Sub1
41171,,0.0,File_77235,0.0,10.558259,100.291312,102.168966,103.072892,104.204259,105.001762,...,93.446136,95.890642,95.166324,96.301512,97.018546,99.777512,100.352210,PRODUCT_3369,2015-04-21 20:07:44,Sub1
41172,,0.0,File_77428,0.0,11.032706,100.639282,101.375000,102.827187,103.433089,105.516328,...,93.633776,95.326541,95.113455,96.384701,97.175039,99.081730,100.975695,PRODUCT_13269,2015-04-21 20:08:46,Sub4


In [78]:
df_result_14= df_result.iloc[130000 : 140000]
for i in range(len(df_result_14)):
    array_productId = df_result_14.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_14.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_3=df_result_detail_3.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_3

,error_code,error_quantity,file_name,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,pressure_104,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_2775,2015-03-30 14:13:35,Sub4
1,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_12788,2015-03-30 14:13:35,Sub4
2,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_2775,2015-03-30 14:15:16,Sub3
3,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_12788,2015-03-30 14:15:16,Sub3
4,,0.0,File_134807,0.0,11.434745,101.389907,102.658200,102.258518,103.709444,105.403732,...,94.969882,94.204441,96.452536,96.909193,98.588181,98.908564,100.870959,PRODUCT_2775,2015-03-30 14:16:09,Sub4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61995,,0.0,File_151489,0.0,11.436110,100.010655,102.670123,103.425331,104.196903,104.833598,...,94.280356,94.807712,95.821341,96.863035,98.892119,98.716388,100.022319,PRODUCT_13539,2015-05-04 03:50:08,Sub1
61996,,0.0,File_151496,0.0,10.739339,101.194131,102.504192,102.099559,103.995793,104.727389,...,93.545331,95.024520,96.649978,97.598723,97.475597,99.862477,99.297082,PRODUCT_13539,2015-05-04 03:57:16,Sub1
61997,,0.0,File_151496,0.0,10.739339,101.194131,102.504192,102.099559,103.995793,104.727389,...,93.545331,95.024520,96.649978,97.598723,97.475597,99.862477,99.297082,PRODUCT_3638,2015-05-04 03:57:16,Sub1
61998,,0.0,File_151578,0.0,10.210652,101.165911,101.556194,103.883543,104.245138,104.790498,...,93.461991,95.254282,95.577846,97.323306,97.495117,98.030918,99.162936,PRODUCT_13539,2015-05-04 03:57:54,Sub3


In [81]:
df_result_15= df_result.iloc[140000 : 150000]
for i in range(len(df_result_15)):
    array_productId = df_result_15.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_15.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_3=df_result_detail_3.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_3

,error_code,error_quantity,file_name,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,pressure_104,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_2775,2015-03-30 14:13:35,Sub4
1,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_12788,2015-03-30 14:13:35,Sub4
2,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_2775,2015-03-30 14:15:16,Sub3
3,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_12788,2015-03-30 14:15:16,Sub3
4,,0.0,File_134807,0.0,11.434745,101.389907,102.658200,102.258518,103.709444,105.403732,...,94.969882,94.204441,96.452536,96.909193,98.588181,98.908564,100.870959,PRODUCT_2775,2015-03-30 14:16:09,Sub4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82436,,0.0,File_23141,0.0,11.746347,101.582943,101.912528,103.981573,103.467694,104.284247,...,93.461116,95.666787,95.557872,97.443983,98.657715,98.323275,100.021178,PRODUCT_3971,2015-05-15 11:25:01,Sub3
82437,,0.0,File_23266,0.0,10.283464,101.008012,101.429042,103.666206,103.018193,104.712874,...,94.137613,95.617534,96.909366,97.472760,97.162208,99.997917,99.653982,PRODUCT_13754,2015-05-15 11:25:52,Sub4
82438,,0.0,File_23266,0.0,10.283464,101.008012,101.429042,103.666206,103.018193,104.712874,...,94.137613,95.617534,96.909366,97.472760,97.162208,99.997917,99.653982,PRODUCT_3971,2015-05-15 11:25:52,Sub4
82439,,0.0,File_23017,0.0,11.314697,101.165105,102.643372,102.506858,104.521689,104.023700,...,94.584496,95.798529,95.502119,96.381576,97.859568,98.365131,99.668210,PRODUCT_13754,2015-05-15 11:26:54,Sub1


In [84]:
df_result_16= df_result.iloc[150000 : 160000]
for i in range(len(df_result_16)):
    array_productId = df_result_16.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_16.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_3=df_result_detail_3.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_3

,error_code,error_quantity,file_name,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,pressure_104,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_2775,2015-03-30 14:13:35,Sub4
1,,0.0,File_134802,0.0,11.273055,101.492190,101.368047,102.723726,104.063162,105.028500,...,93.423053,94.052813,95.402447,97.165398,97.879861,98.937282,100.120553,PRODUCT_12788,2015-03-30 14:13:35,Sub4
2,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_2775,2015-03-30 14:15:16,Sub3
3,,0.0,File_134674,0.0,11.307685,100.194136,102.566619,102.853513,103.258612,105.985070,...,94.428518,95.535477,96.601253,96.161974,98.058054,98.089982,99.429843,PRODUCT_12788,2015-03-30 14:15:16,Sub3
4,,0.0,File_134807,0.0,11.434745,101.389907,102.658200,102.258518,103.709444,105.403732,...,94.969882,94.204441,96.452536,96.909193,98.588181,98.908564,100.870959,PRODUCT_2775,2015-03-30 14:16:09,Sub4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103502,,0.0,File_98708,0.0,10.521467,100.779212,102.559269,103.608969,104.719530,105.322325,...,93.717614,94.111808,96.655715,97.507739,98.512785,98.629958,99.465881,PRODUCT_4280,2015-05-26 13:45:21,Sub4
103503,,0.0,File_98459,0.0,10.385544,100.135895,102.140383,102.852406,104.371568,104.279688,...,93.016602,95.057340,95.875454,96.302110,98.756070,98.085284,100.169190,PRODUCT_13975,2015-05-26 13:46:23,Sub1
103504,,0.0,File_98459,0.0,10.385544,100.135895,102.140383,102.852406,104.371568,104.279688,...,93.016602,95.057340,95.875454,96.302110,98.756070,98.085284,100.169190,PRODUCT_4280,2015-05-26 13:46:23,Sub1
103505,,0.0,File_98595,0.0,11.531121,100.424345,101.598831,102.915182,103.851221,105.658390,...,93.566968,95.512903,95.291297,96.003425,98.703714,99.787735,99.529219,PRODUCT_13975,2015-05-26 13:47:18,Sub3


In [87]:
df_result_detail_4 = pd.DataFrame()
df_result_17= df_result.iloc[160000 : 170000]
for i in range(len(df_result_17)):
    array_productId = df_result_17.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_17.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_4=df_result_detail_4.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_4

,error_code,error_quantity,file_name,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,pressure_104,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,,0.0,File_98714,0.0,10.067135,100.251786,102.109075,102.337241,103.288861,104.998676,...,94.601817,95.777451,96.605357,96.664524,98.820269,98.585281,100.699144,PRODUCT_13975,2015-05-26 13:48:09,Sub4
1,,0.0,File_98714,0.0,10.067135,100.251786,102.109075,102.337241,103.288861,104.998676,...,94.601817,95.777451,96.605357,96.664524,98.820269,98.585281,100.699144,PRODUCT_4280,2015-05-26 13:48:09,Sub4
2,,0.0,File_98464,0.0,11.266624,101.022467,101.719518,103.900713,104.558143,104.379775,...,93.838162,94.655122,95.765429,97.086775,97.942081,99.103275,100.038132,PRODUCT_13975,2015-05-26 13:50:37,Sub1
3,,0.0,File_98464,0.0,11.266624,101.022467,101.719518,103.900713,104.558143,104.379775,...,93.838162,94.655122,95.765429,97.086775,97.942081,99.103275,100.038132,PRODUCT_4280,2015-05-26 13:50:37,Sub1
4,,0.0,File_98601,0.0,10.726714,101.612718,102.266247,103.615553,103.987918,105.762022,...,94.926907,94.641899,96.759829,96.789129,97.217961,99.703635,99.857421,PRODUCT_13975,2015-05-26 13:51:48,Sub3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21151,FQCH,48.0,File_174974,0.0,11.516861,101.563272,101.653002,103.839985,103.520796,105.715567,...,94.949174,95.498413,95.087456,96.008896,97.380402,98.732942,100.857075,PRODUCT_4610,2015-06-07 14:19:37,Sub4
21152,FQCH,48.0,File_174901,0.0,11.677479,100.315661,102.733105,102.888600,104.751781,104.979229,...,93.442057,94.734109,95.573653,96.401550,98.250329,99.092809,99.789182,PRODUCT_14193,2015-06-07 14:20:57,Sub3
21153,FQCH,48.0,File_174901,0.0,11.677479,100.315661,102.733105,102.888600,104.751781,104.979229,...,93.442057,94.734109,95.573653,96.401550,98.250329,99.092809,99.789182,PRODUCT_4610,2015-06-07 14:20:57,Sub3
21154,FQCH,48.0,File_174979,0.0,10.562842,100.077023,101.996686,102.960574,104.572987,105.191882,...,94.532331,95.649251,96.676824,96.330245,98.514316,98.125458,100.491945,PRODUCT_14193,2015-06-07 14:22:11,Sub4


In [90]:
df_result_18= df_result.iloc[170000 : 180000]
for i in range(len(df_result_18)):
    array_productId = df_result_18.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_18.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_4=df_result_detail_4.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_4

,error_code,error_quantity,file_name,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,pressure_104,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,,0.0,File_98714,0.0,10.067135,100.251786,102.109075,102.337241,103.288861,104.998676,...,94.601817,95.777451,96.605357,96.664524,98.820269,98.585281,100.699144,PRODUCT_13975,2015-05-26 13:48:09,Sub4
1,,0.0,File_98714,0.0,10.067135,100.251786,102.109075,102.337241,103.288861,104.998676,...,94.601817,95.777451,96.605357,96.664524,98.820269,98.585281,100.699144,PRODUCT_4280,2015-05-26 13:48:09,Sub4
2,,0.0,File_98464,0.0,11.266624,101.022467,101.719518,103.900713,104.558143,104.379775,...,93.838162,94.655122,95.765429,97.086775,97.942081,99.103275,100.038132,PRODUCT_13975,2015-05-26 13:50:37,Sub1
3,,0.0,File_98464,0.0,11.266624,101.022467,101.719518,103.900713,104.558143,104.379775,...,93.838162,94.655122,95.765429,97.086775,97.942081,99.103275,100.038132,PRODUCT_4280,2015-05-26 13:50:37,Sub1
4,,0.0,File_98601,0.0,10.726714,101.612718,102.266247,103.615553,103.987918,105.762022,...,94.926907,94.641899,96.759829,96.789129,97.217961,99.703635,99.857421,PRODUCT_13975,2015-05-26 13:51:48,Sub3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42098,,0.0,File_56474,0.0,11.617610,100.823870,101.440889,103.750714,104.975681,104.368968,...,94.209517,95.681843,96.681837,96.748574,98.504839,99.449461,99.211866,PRODUCT_5009,2015-06-19 22:12:40,Sub4
42099,,0.0,File_56345,0.0,10.873102,101.260622,101.010253,103.805853,103.199992,104.666435,...,94.170515,95.537417,96.263663,96.230218,98.556980,98.200643,99.750905,PRODUCT_14451,2015-06-19 22:13:42,Sub1
42100,,0.0,File_56345,0.0,10.873102,101.260622,101.010253,103.805853,103.199992,104.666435,...,94.170515,95.537417,96.263663,96.230218,98.556980,98.200643,99.750905,PRODUCT_5009,2015-06-19 22:13:42,Sub1
42101,,0.0,File_56418,0.0,11.570438,100.979232,102.657301,102.883072,103.583243,105.947928,...,93.715279,95.968007,96.867730,96.648318,97.840172,98.543693,100.794404,PRODUCT_14451,2015-06-19 22:14:38,Sub3


In [91]:
df_result_19= df_result.iloc[180000 : 190000]
for i in range(len(df_result_19)):
    array_productId = df_result_19.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_19.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_4=df_result_detail_4.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_4

,error_code,error_quantity,file_name,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,pressure_104,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,,0.0,File_98714,0.0,10.067135,100.251786,102.109075,102.337241,103.288861,104.998676,...,94.601817,95.777451,96.605357,96.664524,98.820269,98.585281,100.699144,PRODUCT_13975,2015-05-26 13:48:09,Sub4
1,,0.0,File_98714,0.0,10.067135,100.251786,102.109075,102.337241,103.288861,104.998676,...,94.601817,95.777451,96.605357,96.664524,98.820269,98.585281,100.699144,PRODUCT_4280,2015-05-26 13:48:09,Sub4
2,,0.0,File_98464,0.0,11.266624,101.022467,101.719518,103.900713,104.558143,104.379775,...,93.838162,94.655122,95.765429,97.086775,97.942081,99.103275,100.038132,PRODUCT_13975,2015-05-26 13:50:37,Sub1
3,,0.0,File_98464,0.0,11.266624,101.022467,101.719518,103.900713,104.558143,104.379775,...,93.838162,94.655122,95.765429,97.086775,97.942081,99.103275,100.038132,PRODUCT_4280,2015-05-26 13:50:37,Sub1
4,,0.0,File_98601,0.0,10.726714,101.612718,102.266247,103.615553,103.987918,105.762022,...,94.926907,94.641899,96.759829,96.789129,97.217961,99.703635,99.857421,PRODUCT_13975,2015-05-26 13:51:48,Sub3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62489,BPAK,6.0,File_134838,0.0,11.306832,100.900426,101.346229,103.580016,104.986737,104.997671,...,93.728881,94.487450,95.089949,96.902537,98.486978,99.588223,99.440835,PRODUCT_14668,2015-07-01 22:34:01,Sub3
62490,BPAK,6.0,File_134922,0.0,11.437973,100.669456,101.689988,103.560647,104.440399,105.870309,...,94.508038,95.203541,95.258402,97.399108,98.642865,99.902115,99.223028,PRODUCT_5346,2015-07-01 22:34:40,Sub4
62491,BPAK,6.0,File_134922,0.0,11.437973,100.669456,101.689988,103.560647,104.440399,105.870309,...,94.508038,95.203541,95.258402,97.399108,98.642865,99.902115,99.223028,PRODUCT_14668,2015-07-01 22:34:40,Sub4
62492,BPAK,6.0,File_134731,0.0,10.565161,100.242223,101.294692,103.757027,104.975282,105.494663,...,93.108716,95.058419,96.028386,96.683726,98.249990,99.781204,99.275564,PRODUCT_5346,2015-07-01 22:35:18,Sub1


In [95]:
df_result_detail_5 = pd.DataFrame()
df_result_20= df_result.iloc[190000 :]
for i in range(len(df_result_20)):
    array_productId = df_result_20.iloc[i , 204].split('-')
    for index in range(len(array_productId)):
        temp = df_result_20.copy()
        temp.iloc[i ,204] = array_productId[index]
        df_result_detail_5=df_result_detail_5.append(temp.iloc[i] , ignore_index = True , sort=False)
df_result_detail_5

,error_code,error_quantity,file_name,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,pressure_104,...,pressure_93,pressure_94,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub
0,,0.0,File_134928,0.0,11.568492,100.793004,101.600471,102.652711,104.881171,104.478788,...,93.202111,95.869355,95.229226,96.631092,97.025077,98.320654,100.815256,PRODUCT_14668,2015-07-01 22:46:44,Sub4
1,,0.0,File_134928,0.0,11.568492,100.793004,101.600471,102.652711,104.881171,104.478788,...,93.202111,95.869355,95.229226,96.631092,97.025077,98.320654,100.815256,PRODUCT_5357,2015-07-01 22:46:44,Sub4
2,,0.0,File_134772,0.0,10.249743,100.716528,101.741005,102.998127,104.289307,105.251566,...,93.026997,95.528168,96.842996,96.628695,97.565786,99.139141,99.221375,PRODUCT_14668,2015-07-01 22:48:02,Sub1
3,,0.0,File_134772,0.0,10.249743,100.716528,101.741005,102.998127,104.289307,105.251566,...,93.026997,95.528168,96.842996,96.628695,97.565786,99.139141,99.221375,PRODUCT_5357,2015-07-01 22:48:02,Sub1
4,,0.0,File_134853,0.0,10.462775,100.518853,101.111361,103.887615,104.632351,104.437267,...,93.032413,94.384533,96.480189,96.631308,98.206825,98.516899,99.471091,PRODUCT_14668,2015-07-01 22:48:57,Sub3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16977,,0.0,File_199530,0.0,11.189193,101.595085,102.393659,103.034215,104.586318,104.168189,...,94.566905,94.326280,96.961336,97.012582,98.468052,99.104916,100.913935,PRODUCT_5604,2015-07-11 18:05:47,Sub4
16978,,0.0,File_199499,0.0,10.818476,100.454595,101.892582,102.099392,103.760500,104.679587,...,94.203233,95.445314,95.607707,97.897887,98.304484,99.446451,99.310463,PRODUCT_14849,2015-07-11 18:06:25,Sub1
16979,,0.0,File_199499,0.0,10.818476,100.454595,101.892582,102.099392,103.760500,104.679587,...,94.203233,95.445314,95.607707,97.897887,98.304484,99.446451,99.310463,PRODUCT_5604,2015-07-11 18:06:25,Sub1
16980,,0.0,File_199509,0.0,11.030629,100.754376,101.875075,102.602257,103.294960,104.551935,...,94.976533,94.028756,96.284959,96.417147,98.490936,99.097503,100.377520,PRODUCT_14849,2015-07-11 18:07:03,Sub2


In [101]:
# df_rs_1 = pd.read_csv('result-ex-final.csv')
# df_rs_2 = pd.read_csv('result-final.csv')
# df_rs_3 = pd.read_csv('result-final-2.csv')
# df_rs_4 = pd.read_csv('test1.csv')
# df_exr_final = pd.concat([df_rs_1,df_rs_2,df_rs_3,df_rs_4])
# df_exr_final.drop('Unnamed: 0' , axis=1 , inplace = True)
# df_exr_final.to_csv('result-ex-final.csv')

In [114]:
# df_exr_final = pd.read_csv('result-ex-final.csv')
# df_exr_final.merge(df_error , on='product_id' , how='left')
# df_exr_final.drop('Unnamed: 0' , axis=1 , inplace = True)
# df_error.drop('Unnamed: 0' , axis = 1 , inplace=True)
df_exr_final = df_exr_final.merge(df_error , on='product_id' , how='left')
df_exr_final

,error_code,error_quantity,file_name,isError,pressure_1,pressure_10,pressure_100,pressure_101,pressure_102,pressure_103,...,pressure_95,pressure_96,pressure_97,pressure_98,pressure_99,product_id,start_at,sub,errors,code
0,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_10165,2014-12-10 00:09:50,Sub1,NaN,NaN
1,BPAK,5.0,File_1,0.0,0.0,10.964720,101.823594,102.623763,103.417978,104.243859,...,95.084069,97.943955,98.684558,98.172818,100.074451,PRODUCT_141,2014-12-10 00:09:50,Sub1,5.0,BPAK
2,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_10165,2014-12-10 00:15:28,Sub1,NaN,NaN
3,BPAK,5.0,File_2,0.0,0.0,10.743145,101.146926,102.322998,102.297234,104.201504,...,96.777099,96.716940,98.017857,99.698420,99.268095,PRODUCT_141,2014-12-10 00:15:28,Sub1,5.0,BPAK
4,BPAK,5.0,File_9,0.0,0.0,11.872049,101.239317,102.399545,103.670321,104.667630,...,95.470922,96.806140,97.468899,98.462504,99.112021,PRODUCT_10165,2014-12-10 00:16:05,Sub2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429916,NaN,0.0,File_199530,NaN,0.0,11.189193,101.595085,102.393659,103.034215,104.586318,...,96.961336,97.012582,98.468052,99.104916,100.913935,PRODUCT_5604,2015-07-11 18:05:47,Sub4,NaN,NaN
429917,NaN,0.0,File_199499,NaN,0.0,10.818476,100.454595,101.892582,102.099392,103.760500,...,95.607707,97.897887,98.304484,99.446451,99.310463,PRODUCT_14849,2015-07-11 18:06:25,Sub1,NaN,NaN
429918,NaN,0.0,File_199499,NaN,0.0,10.818476,100.454595,101.892582,102.099392,103.760500,...,95.607707,97.897887,98.304484,99.446451,99.310463,PRODUCT_5604,2015-07-11 18:06:25,Sub1,NaN,NaN
429919,NaN,0.0,File_199509,NaN,0.0,11.030629,100.754376,101.875075,102.602257,103.294960,...,96.284959,96.417147,98.490936,99.097503,100.377520,PRODUCT_14849,2015-07-11 18:07:03,Sub2,NaN,NaN


In [15]:
X = np.array(df_result['error_quantity'])
y = np.array(df_result['isError'])
X_train , X_test , y_train , y_test = train_test_split(X , y ,test_size=0.2)
logistic_regression = LogisticRegression().fit(X_train.reshape(-1, 1) ,y_train.reshape(-1, 1))
logistic_regression.score(X_test.reshape(-1, 1) ,y_test.reshape(-1, 1))


c:\users\khangdnn.vg\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1.0

In [16]:
result = logistic_regression.predict(np.array([95]).reshape(-1 , 1))
result = 'Error' if result[0] > 0 else 'No Error'
result

'No Error'